In [277]:
import pandas as pd
import micom
from gurobipy import *
import numpy as np
from micom import load_pickle
from micom.media import minimal_medium
from micom.workflows import workflow

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from adjustText import adjust_text
from scipy.stats import ttest_ind

import cobra
# https://resendislab.github.io/micom/media.html

In [278]:
#Verify the qp solver is gurobi
#cobra.util.solver.qp_solvers[1]

In [279]:
#Load same reconstruccion but with abundances of day 6
d = {'id': ['A', 'B', 'C'], 
     'file': ['../SBMLS/cobraA_Jan14_div50_NormalObj_no_PAREN.xml', '../SBMLS/cobraB_Jan8_NormalObj_no_PAREN.xml','../SBMLS/cobraC_Jan8_NormalObj_no_PAREN.xml'],
     'abundance':[0.08955224, 0.3880597, 0.5149254]
    }
df = pd.DataFrame(data=d)
df

,id,file,abundance
0,A,../SBMLS/cobraA_Jan14_div50_NormalObj_no_PAREN...,0.089552
1,B,../SBMLS/cobraB_Jan8_NormalObj_no_PAREN.xml,0.388060
2,C,../SBMLS/cobraC_Jan8_NormalObj_no_PAREN.xml,0.514925


In [280]:
#Load same reconstruccion but with abundances of day 19
d2 = {'id': ['A', 'B', 'C'], 
     'file': ['../SBMLS/cobraA_Jan14_div50_NormalObj_no_PAREN.xml', '../SBMLS/cobraB_Jan8_NormalObj_no_PAREN.xml','../SBMLS/cobraC_Jan8_NormalObj_no_PAREN.xml'],
     'abundance':[0.573913, 0.273913, 0.1521739]
    }
df2 = pd.DataFrame(data=d2)
df2

,id,file,abundance
0,A,../SBMLS/cobraA_Jan14_div50_NormalObj_no_PAREN...,0.573913
1,B,../SBMLS/cobraB_Jan8_NormalObj_no_PAREN.xml,0.273913
2,C,../SBMLS/cobraC_Jan8_NormalObj_no_PAREN.xml,0.152174


In [281]:
#Make community with gurobi
com = micom.Community(df,solver="gurobi")
com2 = micom.Community(df2,solver="gurobi")

In [282]:
#Save to pickle
#com.to_pickle("community_6.pickle")
#com2.to_pickle("community_19.pickle")

In [283]:
print(com.objective.expression)
print(com2.objective.expression)

1.0*community_objective
1.0*community_objective


In [284]:
#For all the exchange reactions of the medium/community check the lower bound (in this case means imports)
for r in com.exchanges:
    if r.lower_bound<0:
        print(r.id,r.lower_bound)

EX_ala_L_m -0.2246
EX_arg_L_m -0.114942532
EX_asp_L_m -0.1514
EX_ca2_m -0.18018
EX_chol_m -0.1428
EX_cl_m -0.14284
EX_cys_L_m -0.1652
EX_fol_m -0.0452
EX_gal_m -0.111
EX_glu_L_m -0.1368
EX_gly_m -0.26666
EX_his_L_m -0.129
EX_ile_L_m -0.1526
EX_inost_m -0.111
EX_leu_L_m -0.1526
EX_lys_L_m -0.1368
EX_met_L_m -0.1342
EX_na1_m -0.344827575
EX_ncam_m -0.16393442
EX_o2_m -0.2
EX_phe_L_m -0.1212
EX_pi_m -0.1408
EX_pnto_R_m -0.04192872
EX_pydxn_m -0.09708738
EX_pyr_m -0.1818
EX_ribflv_m -0.0418
EX_ser_L_m -0.1904
EX_so4_m -0.1666
EX_thm_m -0.0452
EX_thr_L_m -0.16806
EX_trp_L_m -0.098
EX_tyr_L_m -0.1104
EX_val_L_m -0.1708


In [285]:
#Obtain normal optimization without tradeoff (day 6)
sol = com.optimize(fluxes=True, pfba=True)
sol

,abundance,growth_rate,reactions,metabolites
compartments,,,,
A,0.090226,0.000000,1031,889
B,0.390977,0.000000,730,692
C,0.518797,0.060317,706,616
medium,NaN,NaN,106,106


In [286]:
#Obtain normal optimization without tradeoff (day 19)
sol2 = com2.optimize(fluxes=True, pfba=True)
sol2

,abundance,growth_rate,reactions,metabolites
compartments,,,,
A,0.573913,0.000000,1031,889
B,0.273913,0.000000,730,692
C,0.152174,0.205637,706,616
medium,NaN,NaN,106,106


In [287]:
#Obtain optimization with tradeoff=0.7 (day 6)
sol = com.cooperative_tradeoff(fraction=0.7, fluxes=True)
sol

,abundance,growth_rate,reactions,metabolites
compartments,,,,
A,0.090226,0.003776,1031,889
B,0.390977,0.012109,730,692
C,0.518797,0.032440,706,616
medium,NaN,NaN,106,106


In [288]:
#Obtain optimization with tradeoff=0.7 (day 19)
sol2 = com2.cooperative_tradeoff(fraction=0.7, fluxes=True)
sol2

,abundance,growth_rate,reactions,metabolites
compartments,,,,
A,0.573913,0.007702,1031,889
B,0.273913,0.006450,730,692
C,0.152174,0.103291,706,616
medium,NaN,NaN,106,106


In [289]:
rates=[]
med=[]
fluxes=[]

In [290]:
rates.append(sol.members["growth_rate"].copy())
rates.append(sol2.members["growth_rate"].copy())

In [291]:
rates[0]["community"] = sol.growth_rate
rates[1]["community"] = sol2.growth_rate

In [292]:
#the calculated growth_rates for each population and the community per day
rates[0].name = 'day6'
rates[1].name = 'day19'
rates

[compartments
 A            0.003776
 B            0.012109
 C            0.032440
 medium            NaN
 community    0.021905
 Name: day6, dtype: float64,
 compartments
 A            0.007702
 B            0.006450
 C            0.103291
 medium            NaN
 community    0.021905
 Name: day19, dtype: float64]

In [293]:
#Medium that we have
com.medium

{'EX_ala_L_m': 0.2246,
 'EX_arg_L_m': 0.114942532,
 'EX_asp_L_m': 0.1514,
 'EX_ca2_m': 0.18018,
 'EX_chol_m': 0.1428,
 'EX_cl_m': 0.14284,
 'EX_cys_L_m': 0.1652,
 'EX_fol_m': 0.0452,
 'EX_gal_m': 0.111,
 'EX_glu_L_m': 0.1368,
 'EX_gly_m': 0.26666,
 'EX_his_L_m': 0.129,
 'EX_ile_L_m': 0.1526,
 'EX_inost_m': 0.111,
 'EX_leu_L_m': 0.1526,
 'EX_lys_L_m': 0.1368,
 'EX_met_L_m': 0.1342,
 'EX_na1_m': 0.344827575,
 'EX_ncam_m': 0.16393442,
 'EX_o2_m': 0.2,
 'EX_phe_L_m': 0.1212,
 'EX_pi_m': 0.1408,
 'EX_pnto_R_m': 0.04192872,
 'EX_pydxn_m': 0.09708738,
 'EX_pyr_m': 0.1818,
 'EX_ribflv_m': 0.0418,
 'EX_ser_L_m': 0.1904,
 'EX_so4_m': 0.1666,
 'EX_thm_m': 0.0452,
 'EX_thr_L_m': 0.16806,
 'EX_trp_L_m': 0.098,
 'EX_tyr_L_m': 0.1104,
 'EX_val_L_m': 0.1708}

In [294]:
#Minimum medium for the growth (95% efficiency) of the community

In [295]:
med.append(minimal_medium(com, 0.95*sol.growth_rate, exports = True))
med[0].name = 'day6'

med.append(minimal_medium(com2, 0.95*sol2.growth_rate, exports = True))
med[1].name = 'day19'
med

[EX_leu_L_m    0.020867
 EX_chol_m     0.003578
 EX_hco3_m    -0.143777
 EX_lys_L_m    0.012322
 EX_succ_m    -0.022590
 EX_arg_L_m    0.053463
 EX_his_L_m    0.002631
 EX_met_L_m    0.003184
 EX_asp_L_m    0.019287
 EX_ile_L_m    0.006134
 EX_co2_m     -0.056604
 EX_inost_m    0.023842
 EX_pi_m       0.025838
 EX_cys_L_m    0.000969
 EX_o2_m       0.162895
 EX_thr_L_m    0.072399
 EX_trp_L_m    0.000277
 EX_tyr_L_m    0.003323
 EX_urea_m    -0.045987
 EX_ser_L_m    0.023369
 EX_phe_L_m    0.005399
 EX_val_L_m    0.007338
 EX_h_m       -0.101669
 EX_nh4_m     -0.111985
 Name: day6, dtype: float64,
 EX_cys_L_m    0.000969
 EX_leu_L_m    0.020867
 EX_nh4_m     -0.111985
 EX_o2_m       0.162895
 EX_thr_L_m    0.072399
 EX_lys_L_m    0.012322
 EX_succ_m    -0.022590
 EX_trp_L_m    0.000277
 EX_met_L_m    0.003184
 EX_tyr_L_m    0.003323
 EX_urea_m    -0.045987
 EX_ser_L_m    0.023369
 EX_phe_L_m    0.005399
 EX_h2o_m     -0.101669
 EX_pi_m       0.025838
 EX_val_L_m    0.007338
 EX_chol_m 

In [296]:
#Run the cooperative tradeoff again with pfba
sol1 = com.cooperative_tradeoff(fraction=0.7, fluxes=True, pfba=True)
sol21 = com2.cooperative_tradeoff(fraction=0.7, fluxes=True, pfba=True)

In [297]:
#Store all fluxes in variable per day
fluxes.append(sol1.fluxes)
fluxes[0]["sample"] = 'd6'

fluxes.append(sol21.fluxes)
fluxes[1]["sample"] = 'd19'

In [298]:
fluxes[0] #day 6

reaction,10FTHFtm,2AMACHYD,2DR1PP,2HATVACIDhc,2HATVLAChc,2MB2COAc,34HPLFM,34HPPOR,34HPPte,3AIBTm,...,r2472,r2495,r2502,r2513,r2516,r2521,r2537,sink_citr_c,xmpt,sample
compartment,,,,,,,,,,,,,,,,,,,,,
A,NaN,0.087900,NaN,NaN,NaN,NaN,0.0,0.0,0.000603,0.0,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.000174,d6
B,-0.00081,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,-0.000139,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.000557,d6
C,NaN,0.167156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,-0.000450,d6
medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d6


In [299]:
fluxes[1] #day 19

reaction,10FTHFtm,2AMACHYD,2DR1PP,2HATVACIDhc,2HATVLAChc,2MB2COAc,34HPLFM,34HPPOR,34HPPte,3AIBTm,...,r2472,r2495,r2502,r2513,r2516,r2521,r2537,sink_citr_c,xmpt,sample
compartment,,,,,,,,,,,,,,,,,,,,,
A,NaN,0.474544,NaN,NaN,NaN,NaN,0.0,0.0,0.001230,0.0,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.000354,d19
B,-0.000431,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,-0.002577,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.000297,d19
C,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,-0.001871,d19
medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d19


In [37]:
#Quick analysis of different tradeoffs (0 = no tradeoff ---> 9 = 0.1 tradeoff) day 6
sols = com.cooperative_tradeoff(fraction=np.arange(0.1, 1.01, 0.1))
rts = sols.solution.apply(lambda x: x.members.growth_rate)
rts['community'] = sols.solution.apply(lambda x: x.growth_rate)
rts

compartments,A,B,C,medium,community
0,1.256065e-16,2.509274e-17,0.060318,NaN,0.031293
1,2.718779e-03,5.377454e-03,0.049761,NaN,0.028163
2,3.402713e-03,1.103304e-02,0.039348,NaN,0.025034
3,3.776897e-03,1.211058e-02,0.032439,NaN,0.021905
4,3.466169e-03,1.256310e-02,0.026120,NaN,0.018776
5,3.155436e-03,1.301557e-02,0.019801,NaN,0.015646
6,2.625484e-03,1.137710e-02,0.015097,NaN,0.012517
7,1.969113e-03,8.532823e-03,0.011322,NaN,0.009388
8,1.312742e-03,5.688548e-03,0.007548,NaN,0.006259
9,6.563710e-04,2.844274e-03,0.003774,NaN,0.003129


In [38]:
#Quick analysis of different tradeoffs (0 = no tradeoff ---> 9 = 0.1 tradeoff) day 19
sols_19 = com2.cooperative_tradeoff(fraction=np.arange(0.1, 1.01, 0.1))
rts_19 = sols_19.solution.apply(lambda x: x.members.growth_rate)
rts_19['community'] = sols_19.solution.apply(lambda x: x.growth_rate)
rts_19

compartments,A,B,C,medium,community
0,1.185115e-16,2.928831e-16,0.205638,NaN,0.031293
1,4.663440e-03,2.107125e-04,0.167108,NaN,0.028164
2,6.775513e-03,4.952396e-03,0.130043,NaN,0.025034
3,7.702574e-03,6.451187e-03,0.103285,NaN,0.021905
4,1.018736e-02,3.759840e-03,0.078194,NaN,0.018776
5,1.271001e-02,7.495339e-04,0.053535,NaN,0.015646
6,1.273651e-02,2.358551e-03,0.029975,NaN,0.012517
7,1.321387e-02,2.282128e-03,0.007748,NaN,0.009388
8,8.400819e-03,4.009482e-03,0.002227,NaN,0.006259
9,4.200410e-03,2.004741e-03,0.001114,NaN,0.003129


In [152]:
#Obtain all fluxes/reactions related to the below metabolites of day 6
for i in sol1.fluxes:
    if '_pyr' in i:
        print('-----Pyruvate')
        print(i, sol1.fluxes[i],end='\n\n')
    if '_lac' in i:
        print('-----Lactate')
        print(i, sol1.fluxes[i],end='\n\n')
    if '_gal' in i:
        print('-----Galactose')
        print(i, sol1.fluxes[i],end='\n\n')
    if '_o2' in i:
        print('-----Oxygen')
        print(i, sol1.fluxes[i],end='\n\n')
    if '_glu' in i:
        print('-----Glutamate')
        print(i, sol1.fluxes[i],end='\n\n')
    if '_gln' in i:
        print('-----Glutamine')
        print(i, sol1.fluxes[i],end='\n\n')
    if '_co2' in i:
        print('-----Carbon Dio')
        print(i, sol1.fluxes[i],end='\n\n')
    if '_glc' in i:
        print('-----Glucose')
        print(i, sol1.fluxes[i],end='\n\n')

-----Carbon Dio
EX_co2_b compartment
A         0.692947
B         0.000000
C         0.139000
medium         NaN
Name: EX_co2_b, dtype: float64

-----Carbon Dio
EX_co2_e compartment
A         0.692947
B         0.000000
C         0.139000
medium         NaN
Name: EX_co2_e, dtype: float64

-----Carbon Dio
EX_co2_m compartment
A              NaN
B              NaN
C              NaN
medium    0.134634
Name: EX_co2_m, dtype: float64

-----Lactate
EX_gal_b compartment
A        -0.001039
B        -0.003332
C         0.000000
medium         NaN
Name: EX_gal_b, dtype: float64

-----Lactate
EX_gal_e compartment
A        -0.001039
B        -0.003332
C         0.000000
medium         NaN
Name: EX_gal_e, dtype: float64

-----Lactate
EX_gal_m compartment
A              NaN
B              NaN
C              NaN
medium   -0.001397
Name: EX_gal_m, dtype: float64

-----Glucose
EX_glc_D_b compartment
A         NaN
B         0.0
C         NaN
medium    NaN
Name: EX_glc_D_b, dtype: float64

-----Glucose


In [89]:
#Obtain all fluxes/reactions related to the below metabolites of day 6
for i in sol21.fluxes:
    if '_pyr' in i:
        print('-----Pyruvate')
        print(i, sol21.fluxes[i],end='\n\n')
    if '_lac' in i:
        print('-----Lactate')
        print(i, sol21.fluxes[i],end='\n\n')
    if '_gal' in i:
        print('-----Galactose')
        print(i, sol1.fluxes[i],end='\n\n')
    if '_o2' in i:
        print('-----Oxygen')
        print(i, sol21.fluxes[i],end='\n\n')
    if '_glu' in i:
        print('-----Glutamate')
        print(i, sol21.fluxes[i],end='\n\n')
    if '_gln' in i:
        print('-----Glutamine')
        print(i, sol21.fluxes[i],end='\n\n')
    if '_co2' in i:
        print('-----Carbon Dio')
        print(i, sol21.fluxes[i],end='\n\n')
    if '_glc' in i:
        print('-----Glucose')
        print(i, sol21.fluxes[i],end='\n\n')

-----Carbon Dio
EX_co2_b compartment
A         0.868195
B         0.000000
C        -0.429239
medium         NaN
Name: EX_co2_b, dtype: float64

-----Carbon Dio
EX_co2_e compartment
A         0.868195
B         0.000000
C        -0.429239
medium         NaN
Name: EX_co2_e, dtype: float64

-----Carbon Dio
EX_co2_m compartment
A             NaN
B             NaN
C             NaN
medium    0.43295
Name: EX_co2_m, dtype: float64

-----Glucose
EX_glc_D_b compartment
A         NaN
B         0.0
C         NaN
medium    NaN
Name: EX_glc_D_b, dtype: float64

-----Glucose
EX_glc_D_m compartment
A         NaN
B         NaN
C         NaN
medium    0.0
Name: EX_glc_D_m, dtype: float64

-----Glucose
EX_glc_e compartment
A         NaN
B         0.0
C         NaN
medium    NaN
Name: EX_glc_e, dtype: float64

-----Glutamine
EX_gln_L_b compartment
A              NaN
B        -0.003905
C         0.007029
medium         NaN
Name: EX_gln_L_b, dtype: float64

-----Glutamine
EX_gln_L_e compartment
A        

In [ ]:
#Get all reactions that at least have 1 significant flux (dif. 0 and nan)
#You can use ctrl+f to find the reaction you want

In [53]:
#day 6
import math

flux_1=[]

for f in sol1.fluxes:
    if not((math.isnan(sol1.fluxes[f]['A']) or sol1.fluxes[f]['A']==0) 
           and (math.isnan(sol1.fluxes[f]['B']) or sol1.fluxes[f]['B']==0)
           and (math.isnan(sol1.fluxes[f]['C']) or sol1.fluxes[f]['C']==0)
           and (math.isnan(sol1.fluxes[f]['medium']) or sol1.fluxes[f]['medium']==0)):
        print(f, sol1.fluxes[f],end='\n\n')
        flux_1.append((f,sol1.fluxes[f]))      

10FTHFtm compartment
A             NaN
B        -0.00081
C             NaN
medium        NaN
Name: 10FTHFtm, dtype: float64

2AMACHYD compartment
A         0.087900
B         0.000000
C         0.167156
medium         NaN
Name: 2AMACHYD, dtype: float64

34HPPte compartment
A         0.000603
B        -0.000139
C              NaN
medium         NaN
Name: 34HPPte, dtype: float64

3DSPHR compartment
A         0.000066
B         0.000212
C         0.000567
medium         NaN
Name: 3DSPHR, dtype: float64

3HCO3_NAt compartment
A         0.359178
B        -0.061369
C        -0.061383
medium         NaN
Name: 3HCO3_NAt, dtype: float64

AACOAT compartment
A         0.000000
B         0.028198
C         0.009489
medium         NaN
Name: AACOAT, dtype: float64

ACACT10m compartment
A         0.000462
B              NaN
C              NaN
medium         NaN
Name: ACACT10m, dtype: float64

ACACT1r compartment
A         0.000000
B        -0.026716
C        -0.009489
medium         NaN
Name: ACACT1r

Name: COAtm, dtype: float64

CTPS1 compartment
A         0.000183
B         0.000587
C         0.001573
medium         NaN
Name: CTPS1, dtype: float64

CYOOm3 compartment
A              NaN
B              NaN
C         0.150073
medium         NaN
Name: CYOOm3, dtype: float64

CYOR_u10m compartment
A         0.000000
B         0.000000
C         0.300146
medium         NaN
Name: CYOR_u10m, dtype: float64

CYSSNAT4te compartment
A         0.000000
B        -0.145482
C         0.445877
medium         NaN
Name: CYSSNAT4te, dtype: float64

CYSSNAT5tc compartment
A         0.000176
B         0.000000
C         0.001368
medium         NaN
Name: CYSSNAT5tc, dtype: float64

CYTK1 compartment
A         0.0006
B            NaN
C            NaN
medium       NaN
Name: CYTK1, dtype: float64

CYTK7 compartment
A         0.000049
B              NaN
C              NaN
medium         NaN
Name: CYTK7, dtype: float64

DAGK_hs compartment
A        -0.000583
B         0.000212
C         0.000567
medium     

Name: EX_chol_b, dtype: float64

EX_chol_e compartment
A        -0.000624
B        -0.002082
C        -0.005578
medium         NaN
Name: EX_chol_e, dtype: float64

EX_chol_m compartment
A              NaN
B              NaN
C              NaN
medium   -0.003764
Name: EX_chol_m, dtype: float64

EX_co2_b compartment
A         0.692947
B         0.000000
C         0.139000
medium         NaN
Name: EX_co2_b, dtype: float64

EX_co2_e compartment
A         0.692947
B         0.000000
C         0.139000
medium         NaN
Name: EX_co2_e, dtype: float64

EX_co2_m compartment
A              NaN
B              NaN
C              NaN
medium    0.134634
Name: EX_co2_m, dtype: float64

EX_cys_L_b compartment
A        -0.000176
B         0.145482
C        -0.428037
medium         NaN
Name: EX_cys_L_b, dtype: float64

EX_cys_L_e compartment
A        -0.000176
B         0.145482
C        -0.428037
medium         NaN
Name: EX_cys_L_e, dtype: float64

EX_cys_L_m compartment
A            NaN
B           

Name: EX_o2_m, dtype: float64

EX_orot_b compartment
A         0.000000
B        -0.001088
C         0.000820
medium         NaN
Name: EX_orot_b, dtype: float64

EX_orot_e compartment
A         0.000000
B        -0.001088
C         0.000820
medium         NaN
Name: EX_orot_e, dtype: float64

EX_pglyc_hs_b compartment
A        -0.000055
B        -0.000176
C         0.000143
medium         NaN
Name: EX_pglyc_hs_b, dtype: float64

EX_pglyc_hs_e compartment
A        -0.000055
B        -0.000176
C         0.000143
medium         NaN
Name: EX_pglyc_hs_e, dtype: float64

EX_phe_L_b compartment
A        -0.000980
B        -0.003142
C        -0.008417
medium         NaN
Name: EX_phe_L_b, dtype: float64

EX_phe_L_e compartment
A        -0.000980
B        -0.003142
C        -0.008417
medium         NaN
Name: EX_phe_L_e, dtype: float64

EX_phe_L_m compartment
A              NaN
B              NaN
C              NaN
medium   -0.005684
Name: EX_phe_L_m, dtype: float64

EX_pi_b compartment
A         

Name: FORt2m, dtype: float64

FORtr compartment
A        -0.000077
B        -0.000247
C        -0.000662
medium         NaN
Name: FORtr, dtype: float64

FRDPtcr compartment
A         0.000154
B         0.000494
C         0.001324
medium         NaN
Name: FRDPtcr, dtype: float64

FTCD compartment
A         0.090007
B              NaN
C         0.034023
medium         NaN
Name: FTCD, dtype: float64

FTHFLm compartment
A             NaN
B         0.00081
C             NaN
medium        NaN
Name: FTHFLm, dtype: float64

G3PD1 compartment
A         0.045954
B         0.009424
C        -0.026907
medium         NaN
Name: G3PD1, dtype: float64

G5SADrm compartment
A         0.000000
B         0.127467
C         0.000000
medium         NaN
Name: G5SADrm, dtype: float64

GALK compartment
A         0.001039
B         0.003332
C              NaN
medium         NaN
Name: GALK, dtype: float64

GALt2_2 compartment
A              NaN
B         0.003332
C         0.000000
medium         NaN
Name: GALt2

LNSTLSr compartment
A         0.000077
B         0.000247
C         0.000662
medium         NaN
Name: LNSTLSr, dtype: float64

LSTO1r compartment
A              NaN
B         0.000000
C         0.000662
medium         NaN
Name: LSTO1r, dtype: float64

LSTO2r compartment
A         0.000077
B         0.000247
C         0.000000
medium         NaN
Name: LSTO2r, dtype: float64

L_LACtcm compartment
A              NaN
B         0.253455
C              NaN
medium         NaN
Name: L_LACtcm, dtype: float64

L_LACtm compartment
A              NaN
B        -0.253455
C              NaN
medium         NaN
Name: L_LACtm, dtype: float64

MCCCrm compartment
A         0.000000
B              NaN
C         0.056942
medium         NaN
Name: MCCCrm, dtype: float64

MECOALm compartment
A         0.00000
B        -0.00081
C         0.00000
medium        NaN
Name: MECOALm, dtype: float64

MECOAS1m compartment
A         0.00000
B         0.00081
C         0.00000
medium        NaN
Name: MECOAS1m, dtype: flo

Name: PPM, dtype: float64

PRAGSr compartment
A         0.000000
B         0.000000
C         0.004767
medium         NaN
Name: PRAGSr, dtype: float64

PRFGS compartment
A         0.000000
B         0.000000
C         0.004767
medium         NaN
Name: PRFGS, dtype: float64

PRO1xm compartment
A         0.452217
B         0.000000
C         0.000000
medium         NaN
Name: PRO1xm, dtype: float64

PROB0AT2tc compartment
A         0.000000
B              NaN
C         0.013381
medium         NaN
Name: PROB0AT2tc, dtype: float64

PROPAT4te compartment
A         0.453775
B              NaN
C              NaN
medium         NaN
Name: PROPAT4te, dtype: float64

PROtm compartment
A        -0.133994
B        -0.128277
C         0.301501
medium         NaN
Name: PROtm, dtype: float64

PRPNCOAHYDm compartment
A         0.000924
B              NaN
C         0.000000
medium         NaN
Name: PRPNCOAHYDm, dtype: float64

PSDm_hs compartment
A         0.000235
B         0.000671
C         0.001796
m

Name: TYRt, dtype: float64

UDPG4E compartment
A        -0.001039
B        -0.003332
C              NaN
medium         NaN
Name: UDPG4E, dtype: float64

UGLT compartment
A         0.001039
B         0.003332
C              NaN
medium         NaN
Name: UGLT, dtype: float64

UMPK2 compartment
A         0.000000
B         0.000114
C              NaN
medium         NaN
Name: UMPK2, dtype: float64

UMPK4 compartment
A         0.000000
B        -0.034607
C         0.004036
medium         NaN
Name: UMPK4, dtype: float64

UMPK7 compartment
A         0.000000
B         0.035423
C              NaN
medium         NaN
Name: UMPK7, dtype: float64

URCN compartment
A         0.090007
B              NaN
C         0.034023
medium         NaN
Name: URCN, dtype: float64

UREAt5 compartment
A         0.000000
B        -0.127467
C              NaN
medium         NaN
Name: UREAt5, dtype: float64

URIDK2m compartment
A        -0.000049
B              NaN
C        -0.000425
medium         NaN
Name: URIDK2m, 

Name: r0941, dtype: float64

r0962 compartment
A         0.386813
B              NaN
C              NaN
medium         NaN
Name: r0962, dtype: float64

r0975 compartment
A              NaN
B              NaN
C         0.618443
medium         NaN
Name: r0975, dtype: float64

r1005 compartment
A         0.000000
B              NaN
C         0.132849
medium         NaN
Name: r1005, dtype: float64

r1088 compartment
A         0.000000
B        -0.002449
C         0.205584
medium         NaN
Name: r1088, dtype: float64

r1135 compartment
A         0.000077
B              NaN
C              NaN
medium         NaN
Name: r1135, dtype: float64

r1143 compartment
A         0.002639
B         0.008463
C         0.032296
medium         NaN
Name: r1143, dtype: float64

r1144 compartment
A         0.000000
B         0.001289
C         0.295275
medium         NaN
Name: r1144, dtype: float64

r1146 compartment
A         0.000077
B              NaN
C              NaN
medium         NaN
Name: r1146, dty

TypeError: must be real number, not str

In [58]:
#day 19
flux_2=[]

for f in sol21.fluxes:
    if not((math.isnan(sol21.fluxes[f]['A']) or sol21.fluxes[f]['A']==0) 
           and (math.isnan(sol21.fluxes[f]['B']) or sol21.fluxes[f]['B']==0)
           and (math.isnan(sol21.fluxes[f]['C']) or sol21.fluxes[f]['C']==0)
           and (math.isnan(sol21.fluxes[f]['medium']) or sol21.fluxes[f]['medium']==0)):
        print(f, sol21.fluxes[f],end='\n\n')
        flux_2.append((f,sol21.fluxes[f])) 

10FTHFtm compartment
A              NaN
B        -0.000431
C              NaN
medium         NaN
Name: 10FTHFtm, dtype: float64

2AMACHYD compartment
A         0.474544
B         0.000000
C         0.000000
medium         NaN
Name: 2AMACHYD, dtype: float64

34HPPte compartment
A         0.001230
B        -0.002577
C              NaN
medium         NaN
Name: 34HPPte, dtype: float64

3DSPHR compartment
A         0.000135
B         0.000113
C         0.001806
medium         NaN
Name: 3DSPHR, dtype: float64

3HCO3_NAt compartment
A         0.329421
B        -0.040143
C        -0.941717
medium         NaN
Name: 3HCO3_NAt, dtype: float64

AACOAT compartment
A         0.00000
B         0.01502
C         0.03758
medium        NaN
Name: AACOAT, dtype: float64

ACACT10m compartment
A         0.000943
B              NaN
C              NaN
medium         NaN
Name: ACACT10m, dtype: float64

ACACT1r compartment
A         0.00000
B        -0.01423
C        -0.03758
medium        NaN
Name: ACACT1r, dt

Name: CO2ter, dtype: float64

CO2tm compartment
A        -1.350633
B        -0.000789
C         1.279128
medium         NaN
Name: CO2tm, dtype: float64

COAtm compartment
A              NaN
B         0.000000
C         0.012643
medium         NaN
Name: COAtm, dtype: float64

CTPS1 compartment
A         0.000373
B         0.000313
C         0.005007
medium         NaN
Name: CTPS1, dtype: float64

CYOOm3 compartment
A              NaN
B              NaN
C         0.411989
medium         NaN
Name: CYOOm3, dtype: float64

CYOR_u10m compartment
A         0.000000
B         0.000000
C         0.823979
medium         NaN
Name: CYOR_u10m, dtype: float64

CYSSNAT4te compartment
A        -0.323439
B         0.000000
C         2.305427
medium         NaN
Name: CYSSNAT4te, dtype: float64

CYTK1 compartment
A         0.001324
B              NaN
C              NaN
medium         NaN
Name: CYTK1, dtype: float64

DAGK_hs compartment
A        -0.001190
B         0.000113
C         0.001806
medium      

Name: EX_chol_b, dtype: float64

EX_chol_e compartment
A        -0.001272
B        -0.001109
C        -0.017760
medium         NaN
Name: EX_chol_e, dtype: float64

EX_chol_m compartment
A              NaN
B              NaN
C              NaN
medium   -0.003736
Name: EX_chol_m, dtype: float64

EX_co2_b compartment
A         0.868195
B         0.000000
C        -0.429239
medium         NaN
Name: EX_co2_b, dtype: float64

EX_co2_e compartment
A         0.868195
B         0.000000
C        -0.429239
medium         NaN
Name: EX_co2_e, dtype: float64

EX_co2_m compartment
A             NaN
B             NaN
C             NaN
medium    0.43295
Name: EX_co2_m, dtype: float64

EX_cys_L_b compartment
A         0.323439
B         0.000000
C        -2.305427
medium         NaN
Name: EX_cys_L_b, dtype: float64

EX_cys_L_e compartment
A         0.323439
B         0.000000
C        -2.305427
medium         NaN
Name: EX_cys_L_e, dtype: float64

EX_cys_L_m compartment
A            NaN
B            NaN

Name: EX_orot_b, dtype: float64

EX_orot_e compartment
A         0.000000
B        -0.002059
C         0.003705
medium         NaN
Name: EX_orot_e, dtype: float64

EX_pglyc_hs_b compartment
A        -0.000112
B        -0.000094
C         0.000592
medium         NaN
Name: EX_pglyc_hs_b, dtype: float64

EX_pglyc_hs_e compartment
A        -0.000112
B        -0.000094
C         0.000592
medium         NaN
Name: EX_pglyc_hs_e, dtype: float64

EX_phe_L_b compartment
A        -0.001998
B        -0.001674
C        -0.026801
medium         NaN
Name: EX_phe_L_b, dtype: float64

EX_phe_L_e compartment
A        -0.001998
B        -0.001674
C        -0.026801
medium         NaN
Name: EX_phe_L_e, dtype: float64

EX_phe_L_m compartment
A              NaN
B              NaN
C              NaN
medium   -0.005684
Name: EX_phe_L_m, dtype: float64

EX_pi_b compartment
A         0.042523
B         0.000000
C        -0.339102
medium         NaN
Name: EX_pi_b, dtype: float64

EX_pi_e compartment
A         0.

Name: G3PD1, dtype: float64

G5SADrm compartment
A         0.000000
B         0.000394
C         1.199551
medium         NaN
Name: G5SADrm, dtype: float64

GALK compartment
A         0.002119
B         0.001775
C              NaN
medium         NaN
Name: GALK, dtype: float64

GALt2_2 compartment
A              NaN
B         0.001775
C         0.000000
medium         NaN
Name: GALt2_2, dtype: float64

GALt4 compartment
A         0.002119
B              NaN
C              NaN
medium         NaN
Name: GALt4, dtype: float64

GAPD compartment
A         0.140945
B         0.198508
C         0.014212
medium         NaN
Name: GAPD, dtype: float64

GARFT compartment
A         0.000000
B         0.000000
C         0.016251
medium         NaN
Name: GARFT, dtype: float64

GCALDD compartment
A         0.000000
B         0.000000
C         0.232843
medium         NaN
Name: GCALDD, dtype: float64

GHMT2r compartment
A        -0.169120
B         0.000084
C         0.031136
medium         NaN
Name: GHM

MECOALm compartment
A         0.000000
B        -0.000431
C         0.000000
medium         NaN
Name: MECOALm, dtype: float64

MECOAS1m compartment
A         0.000000
B         0.000431
C         0.000000
medium         NaN
Name: MECOAS1m, dtype: float64

METB0AT2tc compartment
A         0.000000
B         0.000987
C         0.015806
medium         NaN
Name: METB0AT2tc, dtype: float64

METB0AT3tc compartment
A         0.001178
B         0.000000
C         0.000000
medium         NaN
Name: METB0AT3tc, dtype: float64

MEVK1c compartment
A         0.000943
B         0.000790
C         0.012643
medium         NaN
Name: MEVK1c, dtype: float64

MGCHrm compartment
A         0.000000
B              NaN
C         0.159281
medium         NaN
Name: MGCHrm, dtype: float64

MMSAD3m compartment
A         0.001885
B              NaN
C         0.000000
medium         NaN
Name: MMSAD3m, dtype: float64

MTHFC compartment
A         0.000515
B              NaN
C         0.029783
medium         NaN
Name: M

Name: PRO1xm, dtype: float64

PROPAT4te compartment
A         0.305684
B              NaN
C              NaN
medium         NaN
Name: PROPAT4te, dtype: float64

PROt4_2_r compartment
A         0.000000
B         0.002266
C        -1.096029
medium         NaN
Name: PROt4_2_r, dtype: float64

PROtm compartment
A        -0.871723
B        -0.000826
C         1.176077
medium         NaN
Name: PROtm, dtype: float64

PRPNCOAHYDm compartment
A         0.001885
B              NaN
C         0.000000
medium         NaN
Name: PRPNCOAHYDm, dtype: float64

PSDm_hs compartment
A         0.000479
B         0.000357
C         0.005720
medium         NaN
Name: PSDm_hs, dtype: float64

PSFLIPm compartment
A         0.000479
B         0.000357
C         0.005720
medium         NaN
Name: PSFLIPm, dtype: float64

PSSA1_hs compartment
A             NaN
B             NaN
C        -0.01776
medium        NaN
Name: PSSA1_hs, dtype: float64

PSSA2_hs compartment
A         0.000052
B        -0.001109
C           

Name: UMPK2, dtype: float64

UMPK4 compartment
A         0.000000
B        -0.018434
C         0.000000
medium         NaN
Name: UMPK4, dtype: float64

UMPK7 compartment
A         0.000000
B         0.020347
C              NaN
medium         NaN
Name: UMPK7, dtype: float64

URCN compartment
A         0.169736
B              NaN
C         0.000000
medium         NaN
Name: URCN, dtype: float64

UREAt compartment
A              NaN
B         0.197671
C              NaN
medium         NaN
Name: UREAt, dtype: float64

UREAt5 compartment
A        -0.086737
B        -0.198065
C              NaN
medium         NaN
Name: UREAt5, dtype: float64

URIDK2m compartment
A        -0.000101
B              NaN
C        -0.001352
medium         NaN
Name: URIDK2m, dtype: float64

VALB0AT2tc compartment
A         0.000000
B         0.000000
C         0.036421
medium         NaN
Name: VALB0AT2tc, dtype: float64

VALB0AT3tc compartment
A         0.002716
B         0.002274
C         0.000000
medium         N

Name: r0940, dtype: float64

r0941 compartment
A         1.347326
B         0.000000
C        -2.843313
medium         NaN
Name: r0941, dtype: float64

r0975 compartment
A              NaN
B              NaN
C         0.530629
medium         NaN
Name: r0975, dtype: float64

r1005 compartment
A         0.000000
B              NaN
C         0.526119
medium         NaN
Name: r1005, dtype: float64

r1088 compartment
A         0.000000
B         0.000000
C         0.501953
medium         NaN
Name: r1088, dtype: float64

r1135 compartment
A         0.000157
B              NaN
C              NaN
medium         NaN
Name: r1135, dtype: float64

r1143 compartment
A         0.005383
B         0.004508
C         0.104998
medium         NaN
Name: r1143, dtype: float64

r1144 compartment
A         0.000000
B         0.000687
C         2.452481
medium         NaN
Name: r1144, dtype: float64

r1146 compartment
A         0.000157
B              NaN
C              NaN
medium         NaN
Name: r1146, dty

TypeError: must be real number, not str

In [62]:
#Number of non-null/non-zero reactions (this means reactions that at least have 1 flux different that 0 or Nan)
print('fluxes day 6:',len(flux_1),' -  fluxes day 19:',len(flux_2))

fluxes day 6: 552  -  fluxes day 19: 550


In [54]:
#Get all exchange fluxes of day 6 from the above subset of reactions
for f in flux_1:
    if 'EX' in f[0]:
        print(f[0],'\n',f[1],end='\n\n')

EX_34hpp_ 
 compartment
A        -0.000603
B         0.000139
C              NaN
medium         NaN
Name: EX_34hpp_, dtype: float64

EX_34hpp_b 
 compartment
A        -0.000603
B         0.000139
C              NaN
medium         NaN
Name: EX_34hpp_b, dtype: float64

EX_Lcystin_b 
 compartment
A         0.000000
B         0.000000
C         0.158231
medium         NaN
Name: EX_Lcystin_b, dtype: float64

EX_Lcystin_m 
 compartment
A             NaN
B             NaN
C             NaN
medium    0.08209
Name: EX_Lcystin_m, dtype: float64

EX_Rtotal_b 
 compartment
A        -0.002025
B              NaN
C         0.000352
medium         NaN
Name: EX_Rtotal_b, dtype: float64

EX_Rtotal_e 
 compartment
A        -0.002025
B              NaN
C         0.000352
medium         NaN
Name: EX_Rtotal_e, dtype: float64

EX_acac_b 
 compartment
A         0.000000
B        -0.029008
C         0.043482
medium         NaN
Name: EX_acac_b, dtype: float64

EX_acac_e 
 compartment
A         0.000000
B       

Name: EX_h_e, dtype: float64

EX_hco3_b 
 compartment
A        -0.690720
B         0.222309
C         0.354664
medium         NaN
Name: EX_hco3_b, dtype: float64

EX_hco3_e 
 compartment
A        -0.690720
B         0.222309
C         0.354664
medium         NaN
Name: EX_hco3_e, dtype: float64

EX_hco3_m 
 compartment
A              NaN
B              NaN
C              NaN
medium    0.208596
Name: EX_hco3_m, dtype: float64

EX_his_L_b 
 compartment
A        -0.090484
B        -0.001531
C        -0.038124
medium         NaN
Name: EX_his_L_b, dtype: float64

EX_his_L_e 
 compartment
A        -0.090484
B        -0.001531
C        -0.038124
medium         NaN
Name: EX_his_L_e, dtype: float64

EX_his_L_m 
 compartment
A              NaN
B              NaN
C              NaN
medium   -0.028541
Name: EX_his_L_m, dtype: float64

EX_ile_L_b 
 compartment
A        -0.001542
B        -0.003464
C        -0.009280
medium         NaN
Name: EX_ile_L_b, dtype: float64

EX_ile_L_e 
 compartment
A     

Name: EX_thr_L_e, dtype: float64

EX_thr_L_m 
 compartment
A              NaN
B              NaN
C              NaN
medium   -0.006891
Name: EX_thr_L_m, dtype: float64

EX_trp_L_b 
 compartment
A        -0.000050
B        -0.000706
C        -0.000432
medium         NaN
Name: EX_trp_L_b, dtype: float64

EX_trp_L_e 
 compartment
A        -0.000050
B        -0.000706
C        -0.000432
medium         NaN
Name: EX_trp_L_e, dtype: float64

EX_trp_L_m 
 compartment
A              NaN
B              NaN
C              NaN
medium   -0.000505
Name: EX_trp_L_m, dtype: float64

EX_tyr_L_b 
 compartment
A         0.000000
B        -0.002073
C        -0.005180
medium         NaN
Name: EX_tyr_L_b, dtype: float64

EX_tyr_L_e 
 compartment
A         0.000000
B        -0.002073
C        -0.005180
medium         NaN
Name: EX_tyr_L_e, dtype: float64

EX_tyr_L_m 
 compartment
A              NaN
B              NaN
C              NaN
medium   -0.003498
Name: EX_tyr_L_m, dtype: float64

EX_urea_b 
 compartme

In [63]:
#Get all exchange fluxes of day 19 from the above subset of reactions
for f in flux_2:
    if 'EX' in f[0]:
        print(f[0],'\n',f[1],end='\n\n')

EX_34hpp_ 
 compartment
A        -0.001230
B         0.002577
C              NaN
medium         NaN
Name: EX_34hpp_, dtype: float64

EX_34hpp_b 
 compartment
A        -0.001230
B         0.002577
C              NaN
medium         NaN
Name: EX_34hpp_b, dtype: float64

EX_HC00250_b 
 compartment
A         0.00827
B             NaN
C             NaN
medium        NaN
Name: EX_HC00250_b, dtype: float64

EX_HC00250_e 
 compartment
A         0.00827
B             NaN
C             NaN
medium        NaN
Name: EX_HC00250_e, dtype: float64

EX_HC00250_m 
 compartment
A              NaN
B              NaN
C              NaN
medium    0.004746
Name: EX_HC00250_m, dtype: float64

EX_Lcystin_b 
 compartment
A         0.000000
B         0.000000
C         0.523854
medium         NaN
Name: EX_Lcystin_b, dtype: float64

EX_Lcystin_m 
 compartment
A              NaN
B              NaN
C              NaN
medium    0.079717
Name: EX_Lcystin_m, dtype: float64

EX_Rtotal_b 
 compartment
A        -0.004130


Name: EX_h_b, dtype: float64

EX_h_e 
 compartment
A         0.548715
B         0.138208
C        -2.318213
medium         NaN
Name: EX_h_e, dtype: float64

EX_hco3_b 
 compartment
A        -0.806997
B         0.120429
C         2.826757
medium         NaN
Name: EX_hco3_b, dtype: float64

EX_hco3_e 
 compartment
A        -0.806997
B         0.120429
C         2.826757
medium         NaN
Name: EX_hco3_e, dtype: float64

EX_his_L_b 
 compartment
A        -0.170710
B        -0.000815
C        -0.013057
medium         NaN
Name: EX_his_L_b, dtype: float64

EX_his_L_e 
 compartment
A        -0.170710
B        -0.000815
C        -0.013057
medium         NaN
Name: EX_his_L_e, dtype: float64

EX_his_L_m 
 compartment
A              NaN
B              NaN
C              NaN
medium   -0.100183
Name: EX_his_L_m, dtype: float64

EX_ile_L_b 
 compartment
A        -0.003146
B        -0.001845
C        -0.029549
medium         NaN
Name: EX_ile_L_b, dtype: float64

EX_ile_L_e 
 compartment
A        -0.

Name: EX_trp_L_e, dtype: float64

EX_trp_L_m 
 compartment
A              NaN
B              NaN
C              NaN
medium   -0.000291
Name: EX_trp_L_m, dtype: float64

EX_tyr_L_b 
 compartment
A         0.000000
B        -0.003606
C        -0.016492
medium         NaN
Name: EX_tyr_L_b, dtype: float64

EX_tyr_L_e 
 compartment
A         0.000000
B        -0.003606
C        -0.016492
medium         NaN
Name: EX_tyr_L_e, dtype: float64

EX_tyr_L_m 
 compartment
A              NaN
B              NaN
C              NaN
medium   -0.003498
Name: EX_tyr_L_m, dtype: float64

EX_urea_b 
 compartment
A         0.086737
B         0.000394
C              NaN
medium         NaN
Name: EX_urea_b, dtype: float64

EX_urea_e 
 compartment
A         0.086737
B         0.000394
C              NaN
medium         NaN
Name: EX_urea_e, dtype: float64

EX_urea_m 
 compartment
A              NaN
B              NaN
C              NaN
medium    0.049887
Name: EX_urea_m, dtype: float64

EX_utp_b 
 compartment
A   

In [ ]:
#Track down most important metabolites and see with reactions are involved and their fluxes

In [164]:
#day 6
track_sol1={}
metabolites=['pyr_','lac_','o2_','glu_','gln_','co2_','glc_','gal_', 'g1p','g6p']

for c in metabolites:
    react_metab=[]
    rm_flux=[]
    for r in com.reactions:
        if c in r.reaction:
            react_metab.append((r.id,r.reaction))

    for r in react_metab:
        #print(r)
        if '_m' not in r[0]:
            rm_flux.append((r,sol1.fluxes[r[0][:-3]]))
            #print(sol1.fluxes[r[0][:-3]],end='\n\n')
        else:
            rm_flux.append((r,sol1.fluxes[r[0]]))
            #print(sol1.fluxes[r[0]],end='\n\n')
    track_sol1[c]=rm_flux

In [165]:
#look at only the specified metabolite (ej. pyr_). The name must be equal to the one on the list above
for t in track_sol1['pyr_']:
    print(t[0])
    print(t[1],end='\n\n')

('EX_pyr_b__A', 'pyr_b__A <=> 0.09022556269772179 pyr_m')
compartment
A        -0.347139
B         0.088057
C        -0.356416
medium         NaN
Name: EX_pyr_b, dtype: float64

('2AMACHYD__A', '2amac_c__A + h2o_c__A --> nh4_c__A + pyr_c__A')
compartment
A         0.087900
B         0.000000
C         0.167156
medium         NaN
Name: 2AMACHYD, dtype: float64

('AGTim__A', 'ala_L_m__A + glx_m__A --> gly_m__A + pyr_m__A')
compartment
A         0.000000
B              NaN
C         0.026907
medium         NaN
Name: AGTim, dtype: float64

('AGTix__A', 'ala_L_x__A + glx_x__A --> gly_x__A + pyr_x__A')
compartment
A         0.0
B         0.0
C         0.0
medium    NaN
Name: AGTix, dtype: float64

('ALATA_L__A', 'akg_c__A + ala_L_c__A <=> glu_L_c__A + pyr_c__A')
compartment
A        -0.480993
B        -0.000949
C              NaN
medium         NaN
Name: ALATA_L, dtype: float64

('D3AIBTm__A', '3aib_D_m__A + pyr_m__A <=> 2mop_m__A + ala_L_m__A')
compartment
A         0.0
B         NaN
C     

Name: r2079, dtype: float64

('EX_pyr_e__B', 'pyr_e__B <=> pyr_b__B')
compartment
A        -0.347139
B         0.088057
C        -0.356416
medium         NaN
Name: EX_pyr_e, dtype: float64

('PYRSMCT1__B', 'na1_e__B + pyr_e__B <=> na1_c__B + pyr_c__B')
compartment
A         0.000000
B        -0.059032
C         0.312934
medium         NaN
Name: PYRSMCT1, dtype: float64

('EX_pyr_b__C', 'pyr_b__C <=> 0.5187970056622756 pyr_m')
compartment
A        -0.347139
B         0.088057
C        -0.356416
medium         NaN
Name: EX_pyr_b, dtype: float64

('2AMACHYD__C', '2amac_c__C + h2o_c__C --> nh4_c__C + pyr_c__C')
compartment
A         0.087900
B         0.000000
C         0.167156
medium         NaN
Name: 2AMACHYD, dtype: float64

('AGTim__C', 'ala_L_m__C + glx_m__C --> gly_m__C + pyr_m__C')
compartment
A         0.000000
B              NaN
C         0.026907
medium         NaN
Name: AGTim, dtype: float64

('AGTix__C', 'ala_L_x__C + glx_x__C --> gly_x__C + pyr_x__C')
compartment
A         0.

In [300]:
#day 19
track_sol21={}
metabolites=['pyr_','lac_','o2_','glu_','gln_','co2_','glc_','gal_', 'g1p','g6p']

for c in metabolites:
    react_metab=[]
    rm_flux=[]
    for r in com2.reactions:
        if c in r.reaction:
            react_metab.append((r.id,r.reaction))

    for r in react_metab:
        #print(r)
        if '_m' not in r[0]:
            rm_flux.append((r,sol21.fluxes[r[0][:-3]]))
            #print(sol1.fluxes[r[0][:-3]],end='\n\n')
        else:
            rm_flux.append((r,sol21.fluxes[r[0]]))
            #print(sol1.fluxes[r[0]],end='\n\n')
    track_sol21[c]=rm_flux

In [301]:
#look at only the specified metabolite (ej. g6p). The name must be equal to the one on the list above
for t in track_sol21['g6p']:
    print(t[0])
    print(t[1],end='\n\n')

('G6PDH2r__A', 'g6p_c__A + nadp_c__A <=> 6pgl_c__A + h_c__A + nadph_c__A')
compartment
A         0.0
B         0.0
C         NaN
medium    NaN
Name: G6PDH2r, dtype: float64

('G6PDH2rer__A', 'g6p_r__A + nadp_r__A <=> 6pgl_r__A + h_r__A + nadph_r__A')
compartment
A         0.0
B         0.0
C         NaN
medium    NaN
Name: G6PDH2rer, dtype: float64

('PGMT__A', 'g1p_c__A <=> g6p_c__A')
compartment
A         0.002119
B         0.001775
C              NaN
medium         NaN
Name: PGMT, dtype: float64

('biomass_carbohydrate__A', '3.87591549295775 g6p_c__A --> biomass_carbohydrate_c__A')
compartment
A         0.000547
B         0.000458
C         0.007334
medium         NaN
Name: biomass_carbohydrate, dtype: float64

('G6PDH1rer__B', 'g6p_r__B + nad_r__B <=> 6pgl_r__B + h_r__B + nadh_r__B')
compartment
A         NaN
B         0.0
C         NaN
medium    NaN
Name: G6PDH1rer, dtype: float64

('G6PDH2r__B', 'g6p_c__B + nadp_c__B <=> 6pgl_c__B + h_c__B + nadph_c__B')
compartment
A         0.0

In [ ]:
#Make knockouts of each population

In [127]:
#"raw" displays new growth_rates, does not compare to previous 
ko6=com.knockout_taxa(fraction=0.7, progress=True, method='raw',diag=False)
ko6

compartments,A,B,C
A,NaN,0.013287,0.023200
B,0.007070,NaN,0.040651
C,0.002426,0.010513,NaN


In [128]:
#"change" displays de differences between the old growth_rates and the new ones (ej. 0.001177 greater than older)
ko6=com.knockout_taxa(fraction=0.7, progress=True, method='change',diag=False)
ko6

compartments,A,B,C
A,NaN,0.001177,-0.009239
B,0.003293,NaN,0.008212
C,-0.001351,-0.001597,NaN


In [129]:
ko19=com2.knockout_taxa(fraction=0.7, progress=True, method='raw',diag=False)
ko19

compartments,A,B,C
A,NaN,0.018966,0.079094
B,0.010628,NaN,0.103863
C,0.006144,0.002932,NaN


In [130]:
ko19=com2.knockout_taxa(fraction=0.7, progress=True, method='change',diag=False)
ko19

compartments,A,B,C
A,NaN,0.012515,-0.024191
B,0.002926,NaN,0.000578
C,-0.001559,-0.003519,NaN


In [132]:
#"relative change" displays rate of change of the old growth_rates  
ko19=com2.knockout_taxa(fraction=0.7, progress=True, method='relative change')
ko19

compartments,A,B,C
A,-1.00000,1.939903,-0.234220
B,0.37984,-1.000000,0.005595
C,-0.20235,-0.545457,-1.000000


In [ ]:
#Calculate elasticities - this means that for each change in "efector" (1 unit) we get a change 
#in the reaction's flux of the value of 'elasticity' 

In [197]:
from micom.elasticity import elasticities, elasticities_by_abundance

In [143]:
eps = elasticities(com, fraction=0.7)

In [144]:
eps_ex = elasticities(com, fraction=0.7,reactions=com.exchanges)

In [150]:
eps

,reaction,taxon,effector,direction,elasticity,type
0,EX_34hpp_b,b__A,EX_arg_L_m,reverse,-0.079937,exchanges
1,EX_3aib_D_b,b__A,EX_arg_L_m,forward,-0.064027,exchanges
2,EX_3aib_b,b__A,EX_arg_L_m,forward,-0.062732,exchanges
3,EX_HC00250_b,b__A,EX_arg_L_m,forward,0.448259,exchanges
4,EX_Lcystin_b,b__A,EX_arg_L_m,forward,-1.084191,exchanges
...,...,...,...,...,...,...
69466,EX_dopa_m,medium,C,forward,0.297323,abundance
69467,EX_ocdcea_m,medium,C,forward,1.044438,abundance
69468,EX_pe_hs_m,medium,C,zero,-0.000002,abundance
69469,EX_tmndnc_m,medium,C,forward,1.025071,abundance


In [147]:
eps_ex.head()

,reaction,taxon,effector,direction,elasticity,type
0,EX_34hpp_m,medium,EX_arg_L_m,forward,-0.079699,exchanges
1,EX_3aib_D_m,medium,EX_arg_L_m,forward,-0.063791,exchanges
2,EX_3aib_m,medium,EX_arg_L_m,forward,-0.062578,exchanges
3,EX_HC00250_m,medium,EX_arg_L_m,forward,0.448253,exchanges
4,EX_Lcystin_m,medium,EX_arg_L_m,forward,-1.190702,exchanges


In [ ]:
#Reactions that export and import from the comunity

In [260]:
#Importa del dia 6
for i in com.exchanges:
    if sol1.fluxes[i.id]['medium']<0:
        print(sol1.fluxes[i.id],end='\n\n')

compartment
A              NaN
B              NaN
C              NaN
medium   -0.057706
Name: EX_arg_L_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium   -0.020302
Name: EX_asp_L_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium   -0.003764
Name: EX_chol_m, dtype: float64

compartment
A            NaN
B            NaN
C            NaN
medium   -0.1652
Name: EX_cys_L_m, dtype: float64

compartment
A            NaN
B            NaN
C            NaN
medium   -0.0349
Name: EX_fol_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium   -0.001397
Name: EX_gal_m, dtype: float64

compartment
A            NaN
B            NaN
C            NaN
medium   -0.1368
Name: EX_glu_L_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium   -0.021174
Name: EX_gly_m, dtype: float64

compartment
A              NaN
B              NaN
C    

In [262]:
#Exporta del dia 6
for i in com.exchanges:
    if sol1.fluxes[i.id]['medium']>0:
        print(sol1.fluxes[i.id],end='\n\n')

compartment
A             NaN
B             NaN
C             NaN
medium    0.08209
Name: EX_Lcystin_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium    0.011217
Name: EX_acac_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium    0.000317
Name: EX_acetone_m, dtype: float64

compartment
A             NaN
B             NaN
C             NaN
medium    0.01863
Name: EX_ala_L_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium    0.134634
Name: EX_co2_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium    0.003938
Name: EX_fum_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium    0.208596
Name: EX_hco3_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium    0.347188
Name: EX_nh4_m, dtype: float64

compartment
A            NaN
B      

In [261]:
#Importa del dia 19
for i in com2.exchanges:
    if sol21.fluxes[i.id]['medium']<0:
        print(sol21.fluxes[i.id],end='\n\n')

compartment
A              NaN
B              NaN
C              NaN
medium   -0.057757
Name: EX_arg_L_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium   -0.020302
Name: EX_asp_L_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium   -0.003736
Name: EX_chol_m, dtype: float64

compartment
A            NaN
B            NaN
C            NaN
medium   -0.1652
Name: EX_cys_L_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium   -0.001703
Name: EX_gal_m, dtype: float64

compartment
A            NaN
B            NaN
C            NaN
medium   -0.1368
Name: EX_glu_L_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium   -0.071144
Name: EX_gly_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium   -0.100183
Name: EX_his_L_m, dtype: float64

compartment
A              NaN
B             

In [263]:
#Exporta del dia 19
for i in com2.exchanges:
    if sol21.fluxes[i.id]['medium']>0:
        print(sol21.fluxes[i.id],end='\n\n')

compartment
A              NaN
B              NaN
C              NaN
medium    0.004746
Name: EX_HC00250_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium    0.079717
Name: EX_Lcystin_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium    0.012363
Name: EX_acac_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium    0.000118
Name: EX_acetone_m, dtype: float64

compartment
A             NaN
B             NaN
C             NaN
medium    0.48451
Name: EX_ala_L_m, dtype: float64

compartment
A             NaN
B             NaN
C             NaN
medium    0.43295
Name: EX_co2_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium    0.000008
Name: EX_for_m, dtype: float64

compartment
A              NaN
B              NaN
C              NaN
medium    0.003938
Name: EX_fum_m, dtype: float64

compartment
A              NaN
B 

In [ ]:
#Reactions that export and import per population (Example)

In [272]:
#Importa del dia 6
for i in sol1.fluxes:
    if ('EX' in i) and sol1.fluxes[i]['A']<0: 
        print(sol1.fluxes[i],end='\n\n')

compartment
A        -0.000603
B         0.000139
C              NaN
medium         NaN
Name: EX_34hpp_, dtype: float64

compartment
A        -0.000603
B         0.000139
C              NaN
medium         NaN
Name: EX_34hpp_b, dtype: float64

compartment
A        -0.002025
B              NaN
C         0.000352
medium         NaN
Name: EX_Rtotal_b, dtype: float64

compartment
A        -0.002025
B              NaN
C         0.000352
medium         NaN
Name: EX_Rtotal_e, dtype: float64

compartment
A        -0.000253
B        -0.000810
C         0.000654
medium         NaN
Name: EX_aicar_b, dtype: float64

compartment
A        -0.000253
B        -0.000810
C         0.000654
medium         NaN
Name: EX_aicar_e, dtype: float64

compartment
A        -0.001357
B        -0.131818
C        -0.011654
medium         NaN
Name: EX_arg_L_b, dtype: float64

compartment
A        -0.001357
B        -0.131818
C        -0.011654
medium         NaN
Name: EX_arg_L_e, dtype: float64

compartment
A        -0

In [273]:
#Export del dia 6
for i in sol1.fluxes:
    if ('EX' in i) and sol1.fluxes[i]['A']>0: 
        print(sol1.fluxes[i],end='\n\n')

compartment
A         0.347602
B              NaN
C        -0.060452
medium         NaN
Name: EX_akg_b, dtype: float64

compartment
A         0.347602
B              NaN
C        -0.060452
medium         NaN
Name: EX_akg_e, dtype: float64

compartment
A         0.479084
B        -0.005439
C        -0.043310
medium         NaN
Name: EX_ala_L_b, dtype: float64

compartment
A         0.479084
B        -0.005439
C        -0.043310
medium         NaN
Name: EX_ala_L_e, dtype: float64

compartment
A         0.003776
B         0.012109
C         0.032440
medium         NaN
Name: EX_biomass_c, dtype: float64

compartment
A         0.692947
B         0.000000
C         0.139000
medium         NaN
Name: EX_co2_b, dtype: float64

compartment
A         0.692947
B         0.000000
C         0.139000
medium         NaN
Name: EX_co2_e, dtype: float64

compartment
A         0.000077
B        -0.000018
C              NaN
medium         NaN
Name: EX_for_b, dtype: float64

compartment
A         0.000077
B

In [ ]:
#Obtener que reacciones comparten las poblaciones en COM (Se tiene que hacer para com2)

In [226]:
rids = [(r.global_id, r.community_id) for r in com.reactions]
rlist = pd.DataFrame(rids, columns=["reaction", "id"])
rlist["value"] = 1
rlist = rlist.pivot_table(values="value", index="id", columns="reaction")
inclusion = rlist.fillna(0).astype(int).values

In [227]:
rlist

reaction,10FTHFtm,2AMACHYD,2DR1PP,2HATVACIDhc,2HATVLAChc,2MB2COAc,34HPLFM,34HPPOR,34HPPte,3AIBTm,...,r2438,r2472,r2495,r2502,r2513,r2516,r2521,r2537,sink_citr_c,xmpt
id,,,,,,,,,,,,,,,,,,,,,
A,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0
B,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0
C,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0
medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [221]:
all_pop_reac=[]
for i in rlist:
    if rlist[i]['A']==1 and rlist[i]['B']==1 and rlist[i]['C']==1:
        all_pop_reac.append(i)

len(all_pop_reac)

377

In [228]:
A_excl_reac=[]
for i in rlist:
    if rlist[i]['A']==1 and rlist[i]['B']!=1 and rlist[i]['C']!=1:
        A_excl_reac.append(i)

len(A_excl_reac)

409

In [229]:
B_excl_reac=[]
for i in rlist:
    if rlist[i]['A']!=1 and rlist[i]['B']==1 and rlist[i]['C']!=1:
        B_excl_reac.append(i)

len(B_excl_reac)

185

In [230]:
C_excl_reac=[]
for i in rlist:
    if rlist[i]['A']!=1 and rlist[i]['B']!=1 and rlist[i]['C']==1:
        C_excl_reac.append(i)

len(C_excl_reac)

150

In [231]:
AB_excl_reac=[]
for i in rlist:
    if rlist[i]['A']==1 and rlist[i]['B']==1 and rlist[i]['C']!=1:
        AB_excl_reac.append(i)

len(AB_excl_reac)

117

In [232]:
AC_excl_reac=[]
for i in rlist:
    if rlist[i]['A']==1 and rlist[i]['B']!=1 and rlist[i]['C']==1:
        AC_excl_reac.append(i)

len(AC_excl_reac)

128

In [233]:
BC_excl_reac=[]
for i in rlist:
    if rlist[i]['A']!=1 and rlist[i]['B']==1 and rlist[i]['C']==1:
        BC_excl_reac.append(i)

len(BC_excl_reac)

51

In [240]:
m_excl_reac=[]
for i in rlist:
    if rlist[i]['A']!=1 and rlist[i]['B']!=1 and rlist[i]['C']!=1:
        m_excl_reac.append(i)

len(m_excl_reac)

106

In [ ]:
#Obtener distancias en la communidades segun sus reacciones solamente

In [247]:
inclusion

array([[0, 1, 0, ..., 0, 0, 1],
       [1, 1, 1, ..., 0, 1, 1],
       [0, 1, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [248]:
micom.algorithms.euclidean(inclusion)

array([[ 0.        , 27.80287755, 26.96293753, 33.7194306 ],
       [27.80287755,  0.        , 24.08318916, 28.91366459],
       [26.96293753, 24.08318916,  0.        , 28.4956137 ],
       [33.7194306 , 28.91366459, 28.4956137 ,  0.        ]])

In [249]:
micom.algorithms.jaccard(inclusion)

array([[0.        , 0.6101026 , 0.5900974 , 1.        ],
       [0.6101026 , 0.        , 0.57539683, 1.        ],
       [0.5900974 , 0.57539683, 0.        , 1.        ],
       [1.        , 1.        , 1.        , 0.        ]])

In [166]:
import cobra
recon = cobra.io.read_sbml_model("../recon22.xml")

In [179]:
import pandas as pd
G = pd.read_csv("../HugoV2.csv")

,Hugo,Name,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,HGNC:5,A1BG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HGNC:37133,A1BG-AS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HGNC:24086,A1CF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HGNC:7,A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HGNC:27057,A2M-AS1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42299,HGNC:25820,ZYG11B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42300,HGNC:13200,ZYX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42301,HGNC:51695,ZYXP1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42302,HGNC:29027,ZZEF1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [192]:
len(recon.genes)

1675

In [191]:
x={}
c=0
for i in G['Hugo']:
    x[i]=G['Name'][c]
    c+=1

len(x)

42304

In [195]:
cc=0
for i in recon.genes:
    cc+=1
    if str(i) in x:
        print(str(i)+','+str(x[str(i)]))


HGNC:550,AOC3
HGNC:80,AOC1
HGNC:549,AOC2
HGNC:2602,CYP24A1
HGNC:2606,CYP27B1
HGNC:10691,SDS
HGNC:11455,SULT1A3
HGNC:6535,LDHA
HGNC:6541,LDHB
HGNC:28335,LDHAL6A
HGNC:21481,LDHAL6B
HGNC:30866,UEVLD
HGNC:6544,LDHC
HGNC:2911,DLST
HGNC:2898,DLD
HGNC:21350,PDHX
HGNC:8124,OGDH
HGNC:252,ADH4
HGNC:250,ADH1B
HGNC:253,ADH5
HGNC:255,ADH6
HGNC:16354,ADHFE1
HGNC:251,ADH1C
HGNC:256,ADH7
HGNC:249,ADH1A
HGNC:2228,COMT
HGNC:405,ALDH3A1
HGNC:411,ALDH3B2
HGNC:409,ALDH1A3
HGNC:410,ALDH3B1
HGNC:5147,HPD
HGNC:4021,KDSR
HGNC:4796,HAAO
HGNC:23408,ECHDC2
HGNC:1564,KYAT1
HGNC:17929,AADAT
HGNC:2719,DDC
HGNC:4908,HIBCH
HGNC:6833,MAOA
HGNC:6834,MAOB
HGNC:124,ACP5
HGNC:29609,ACP6
HGNC:123,ACP2
HGNC:125,ACP3
HGNC:4093,GAD2
HGNC:4092,GAD1
HGNC:18966,CSAD
HGNC:4432,GOT1
HGNC:4433,GOT2
HGNC:406,ALDH4A1
HGNC:11453,SULT1A1
HGNC:11454,SULT1A2
HGNC:11458,SULT2A1
HGNC:553,AOX1
HGNC:404,ALDH2
HGNC:407,ALDH1B1
HGNC:6059,IDO1
HGNC:6827,MAN2C1
HGNC:6826,MAN2B1
HGNC:18149,A4GALT
HGNC:17968,A4GNT
HGNC:21298,AACS
HGNC:79,ABO
HGNC:2

HGNC:3515,EXTL1
HGNC:321,AGL
HGNC:21016,LGSN
HGNC:4341,GLUL
HGNC:9723,PYGB
HGNC:9725,PYGL
HGNC:9726,PYGM
HGNC:4312,GCLM
HGNC:4311,GCLC
HGNC:4335,GLUD1
HGNC:4336,GLUD2
HGNC:4331,GLS
HGNC:29570,GLS2
HGNC:9238,PPAT
HGNC:4189,GCDH
HGNC:19708,LDHD
HGNC:4809,HAO1
HGNC:4810,HAO2
HGNC:4175,GATM
HGNC:4570,GRHPR
HGNC:4291,GK2
HGNC:4289,GK
HGNC:4805,HAGH
HGNC:14177,HAGHL
HGNC:4369,GMDS
HGNC:4376,GMPR
HGNC:4377,GMPR2
HGNC:4378,GMPS
HGNC:8891,PGD
HGNC:4415,GNMT
HGNC:24865,GPAM
HGNC:28157,GPAT3
HGNC:27168,GPAT2
HGNC:23213,PIGW
HGNC:25712,PGAP1
HGNC:4623,GSR
HGNC:4553,GPX1
HGNC:9353,PRDX2
HGNC:4556,GPX4
HGNC:9352,PRDX1
HGNC:4554,GPX2
HGNC:4555,GPX3
HGNC:4559,GPX7
HGNC:4558,GPX6
HGNC:9354,PRDX3
HGNC:4624,GSS
HGNC:4193,GCH1
HGNC:7944,NPR2
HGNC:4689,GUCY2D
HGNC:4688,GUCY2C
HGNC:7943,NPR1
HGNC:4691,GUCY2F
HGNC:4686,GUCY1B2
HGNC:4685,GUCY1A1
HGNC:4687,GUCY1B1
HGNC:4684,GUCY1A2
HGNC:4212,GDA
HGNC:5157,HPRT1
HGNC:1372,CA14
HGNC:1371,CA12
HGNC:1373,CA2
HGNC:1374,CA3
HGNC:1380,CA6
HGNC:1381,CA7
HGNC:14914,CA1

HGNC:5389,IDS
HGNC:19419,HS3ST5
HGNC:5200,HS3ST4
HGNC:5194,HS3ST1
HGNC:5195,HS3ST2
HGNC:5196,HS3ST3A1
HGNC:5198,HS3ST3B1
HGNC:14178,HS3ST6
HGNC:17422,CHST11
HGNC:17423,CHST12
HGNC:21755,CHST13
HGNC:24464,CHST14
HGNC:714,ARSB
HGNC:1972,CHST4
HGNC:1973,CHST5
HGNC:1970,CHST2
HGNC:6938,CHST6
HGNC:1971,CHST3
HGNC:1969,CHST1
HGNC:13817,CHST7
HGNC:5201,HS6ST1
HGNC:19134,HS6ST3
HGNC:19133,HS6ST2
HGNC:4422,GNS
HGNC:17366,AASS
HGNC:26309,COMTD1
HGNC:5028,HNMT
HGNC:10536,SARDH
HGNC:17720,SGPP1
HGNC:11184,SORD
HGNC:22204,AHCYL2
HGNC:18161,SCLY
HGNC:19685,SEPHS1
HGNC:19686,SEPHS2
HGNC:16253,SPTLC3
HGNC:11278,SPTLC2
HGNC:11277,SPTLC1
HGNC:3465,ESD
HGNC:10817,SGPL1
HGNC:7759,NEU2
HGNC:21328,NEU4
HGNC:10872,ST3GAL5
HGNC:18859,SPHK2
HGNC:11240,SPHK1
HGNC:14240,SMPD3
HGNC:32949,SMPD4
HGNC:23764,ENPP7
HGNC:11120,SMPD1
HGNC:11121,SMPD2
HGNC:29799,SGMS1
HGNC:11177,SOAT1
HGNC:11296,SRM
HGNC:11179,SOD1
HGNC:11181,SOD3
HGNC:11180,SOD2
HGNC:11257,SPR
HGNC:11123,SMS
HGNC:11279,SQLE
HGNC:3629,FDFT1
HGNC:11284,SR

HGNC:19119,SLC5A8
HGNC:10914,SLC12A6
HGNC:10913,SLC12A4
HGNC:10915,SLC12A7
HGNC:13818,SLC12A5
HGNC:17123,SLC9A7
HGNC:20728,SLC9A8
HGNC:18057,SLC22A10
HGNC:26516,CYB5D1
HGNC:643,AQP9
HGNC:7700,NDUFB5
HGNC:7459,MT-ND4
HGNC:7692,NDUFA8
HGNC:7699,NDUFB4
HGNC:7702,NDUFB7
HGNC:7711,NDUFS4
HGNC:7714,NDUFS7
HGNC:7458,MT-ND3
HGNC:30242,TUSC3
HGNC:17194,NDUFA13
HGNC:7687,NDUFA4
HGNC:20372,NDUFB11
HGNC:7708,NDUFS2
HGNC:7698,NDUFB3
HGNC:7715,NDUFS8
HGNC:7685,NDUFA2
HGNC:7455,MT-ND1
HGNC:7716,NDUFV1
HGNC:7460,MT-ND4L
HGNC:7686,NDUFA3
HGNC:7695,NDUFB1
HGNC:7461,MT-ND5
HGNC:7691,NDUFA7
HGNC:7713,NDUFS6
HGNC:7684,NDUFA10
HGNC:20371,NDUFA11
HGNC:7703,NDUFB8
HGNC:7693,NDUFA9
HGNC:7717,NDUFV2
HGNC:7690,NDUFA6
HGNC:7701,NDUFB6
HGNC:7683,NDUFA1
HGNC:7706,NDUFC2
HGNC:7456,MT-ND2
HGNC:7705,NDUFC1
HGNC:7719,NDUFV3
HGNC:7697,NDUFB2
HGNC:7710,NDUFS3
HGNC:7707,NDUFS1
HGNC:7704,NDUFB9
HGNC:7688,NDUFA5
HGNC:7696,NDUFB10
HGNC:7462,MT-ND6
HGNC:7712,NDUFS5
HGNC:7694,NDUFAB1
HGNC:23987,NDUFA12
HGNC:13811,SLC4A10
HGNC: